In [1]:
import random
from collections import Counter

# Card Ranks and Suits

In [2]:
SUITS = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
HAND_RANKS = ['High Card', 'One Pair', 'Two Pair', 'Three of a Kind', 'Straight', 'Flush', 'Full House', 'Four of a Kind', 'Straight Flush', 'Royal Flush']

# Creating the Deck

In [3]:
def create_deck():
    return [(rank, suit) for suit in SUITS for rank in RANKS]

# Shuffling the Deck

In [4]:
def shuffle_deck(deck):
    random.shuffle(deck)

# Dealing cards to *players*

In [5]:
def deal_cards(deck, num_players=6):
    hands = {f'Player {i+1}': [] for i in range(num_players)}
    for i in range(num_players):
        hands[f'Player {i+1}'].append(deck.pop())
        hands[f'Player {i+1}'].append(deck.pop())
    return hands, deck

# Dealing community cards

In [6]:
def deal_community_cards(deck):
    return [deck.pop() for _ in range(5)]

# Helper function to determine if hand is a Flush

In [7]:
def is_flush(cards):
    suits = [card[1] for card in cards]
    return len(set(suits)) == 1

# Helper function to determine if hand is a Straight

In [8]:
def is_straight(cards):
    values = sorted([RANKS.index(card[0]) for card in cards])
    if values == list(range(min(values), max(values) + 1)):
        return True
    if set(values) == {0, 1, 2, 3, 12}:
        return True
    return False

# Function to evaluate the best hand


In [9]:
def evaluate_hand(cards):
    values = [card[0] for card in cards]
    counts = Counter(values)
    flush = is_flush(cards)
    straight = is_straight(cards)
    if straight and flush:
        if 'A' in values and 'K' in values:
            return "Royal Flush"
        return "Straight Flush"
    if 4 in counts.values():
        return "Four of a Kind"
    if 3 in counts.values() and 2 in counts.values():
        return "Full House"
    if flush:
        return "Flush"
    if straight:
        return "Straight"
    if 3 in counts.values():
        return "Three of a Kind"
    if list(counts.values()).count(2) == 2:
        return "Two Pair"
    if 2 in counts.values():
        return "One Pair"
    return "High Card"

# Pot odds calculation

In [10]:
def pot_odds(call, pot):
    return call / pot if pot > 0 else 0

# AI decision-making function using Expectiminimax


In [11]:
def expectiminimax_decision(hand, community_cards, deck, current_bet, pot, depth=2):
    def simulate_opponent_response(ai_strength, odds):
        if ai_strength > 6:
            return ['call', 'raise']
        elif ai_strength > 3:
            return ['call', 'fold']
        else:
            return ['fold', 'bluff'] if odds > 0.3 else ['call']

    def evaluate_terminal(hand, community_cards):
        full_hand = hand + community_cards
        strength = evaluate_hand(full_hand)
        values = {
            "High Card": 1, "One Pair": 2, "Two Pair": 3, "Three of a Kind": 4,
            "Straight": 5, "Flush": 6, "Full House": 7, "Four of a Kind": 8,
            "Straight Flush": 9, "Royal Flush": 10
        }
        return values[strength]

    def expectiminimax(hand, community, deck, depth, is_ai_turn):
        if depth == 0 or len(community) == 5:
            return evaluate_terminal(hand, community)

        if is_ai_turn:
            fold_val = 0
            call_val = expectiminimax(hand, community, deck[:], depth-1, False)
            raise_val = call_val + 1
            return max(fold_val, call_val, raise_val)
        else:
            odds = pot_odds(current_bet, pot)
            ai_strength = evaluate_terminal(hand, community)
            moves = simulate_opponent_response(ai_strength, odds)
            values = []
            for move in moves:
                if move == 'fold':
                    values.append(1)
                elif move == 'call':
                    values.append(expectiminimax(hand, community, deck[:], depth-1, True))
                elif move == 'raise':
                    values.append(expectiminimax(hand, community, deck[:], depth-1, True) - 1)
                elif move == 'bluff':
                    values.append(random.choice([0, 1]))
            return sum(values) / len(values)

    simulations = 5
    scores = []
    for _ in range(simulations):
        sim_community = community_cards[:]
        sim_deck = deck[:]
        while len(sim_community) < 5 and sim_deck:
            sim_community.append(sim_deck.pop(random.randint(0, len(sim_deck) - 1)))
        score = expectiminimax(hand, sim_community, sim_deck, depth, True)
        scores.append(score)

    avg_score = sum(scores) / len(scores)
    if avg_score > 7:
        return 'raise'
    elif avg_score > 4:
        return 'call'
    elif avg_score > 2:
        return 'bluff'
    else:
        return 'fold'

# Betting mechanics

In [12]:
def betting_round(hands, pot, deck, community_cards, round_name="Betting", min_bet=10):
    print(f"\n--- {round_name.upper()} ROUND ---")
    active_players = {player: {'hand': hand, 'status': 'active', 'bet': 0} for player, hand in hands.items()}
    player_order = list(active_players.keys())
    current_bet = 0
    index = 0
    players_in_round = len(player_order)

    while True:
        player = player_order[index]
        player_data = active_players[player]

        if player_data['status'] != 'active':
            index = (index + 1) % players_in_round
            continue

        print(f"\n{player}'s turn. Current bet to call: {current_bet}")
        print(f"{player}'s hand: {player_data['hand']}")
        print(f"Pot: {pot}, Your bet so far: {player_data['bet']}")

        if player == 'Player 1':
            valid_actions = ["fold"]
            if player_data['bet'] == current_bet:
                valid_actions += ["check", "bet"]
            else:
                valid_actions += ["call", "raise"]
            action = input(f"Choose action ({', '.join(valid_actions)}): ").lower()

            if action == "fold":
                active_players[player]['status'] = 'folded'
            elif action == "check":
                pass
            elif action == "call":
                call_amount = current_bet - player_data['bet']
                active_players[player]['bet'] += call_amount
                pot += call_amount
            elif action == "bet":
                bet_amount = int(input("Enter bet amount: "))
                current_bet = bet_amount
                active_players[player]['bet'] = bet_amount
                pot += bet_amount
            elif action == "raise":
                raise_amount = int(input("Enter raise amount: "))
                total_bet = current_bet + raise_amount
                raise_diff = total_bet - player_data['bet']
                active_players[player]['bet'] = total_bet
                pot += raise_diff
                current_bet = total_bet
        else:
            decision = expectiminimax_decision(player_data['hand'], community_cards, deck[:], current_bet, pot)
            if decision == 'fold':
                print(f"{player} folds.")
                active_players[player]['status'] = 'folded'
            elif decision == 'call':
                call_amount = current_bet - player_data['bet']
                active_players[player]['bet'] += call_amount
                pot += call_amount
                print(f"{player} calls {call_amount}.")
            elif decision == 'raise':
                raise_amount = 10
                total_bet = current_bet + raise_amount
                raise_diff = total_bet - player_data['bet']
                active_players[player]['bet'] = total_bet
                pot += raise_diff
                current_bet = total_bet
                print(f"{player} raises to {total_bet}.")
            elif decision == 'bluff':
                bluff_amount = current_bet + 15
                raise_diff = bluff_amount - player_data['bet']
                active_players[player]['bet'] = bluff_amount
                pot += raise_diff
                current_bet = bluff_amount
                print(f"{player} bluffs with a raise to {bluff_amount}.")

        all_equal = all(
            p['status'] != 'active' or p['bet'] == current_bet
            for p in active_players.values()
        )
        active_count = sum(1 for p in active_players.values() if p['status'] == 'active')
        if all_equal and active_count > 1:
            break
        if active_count <= 1:
            break

        index = (index + 1) % players_in_round

    hands = {
        player: data['hand']
        for player, data in active_players.items()
        if data['status'] == 'active'
    }

    return hands, pot

# Show community cards and display actions

In [13]:
def show_community_cards(community_cards, round_stage):
    print(f"\n{round_stage} Community Cards: {community_cards}")

# Game setup and flow

In [14]:
def play_game():
    deck = create_deck()
    shuffle_deck(deck)
    hands, deck = deal_cards(deck, num_players=6)
    community_cards = []
    pot = 0

    hands, pot = betting_round(hands, pot, deck, community_cards, round_name="Preflop")
    community_cards += [deck.pop() for _ in range(3)]
    show_community_cards(community_cards, "Flop")
    hands, pot = betting_round(hands, pot, deck, community_cards, round_name="Flop")

    community_cards.append(deck.pop())
    show_community_cards(community_cards, "Turn")
    hands, pot = betting_round(hands, pot, deck, community_cards, round_name="Turn")

    community_cards.append(deck.pop())
    show_community_cards(community_cards, "River")
    hands, pot = betting_round(hands, pot, deck, community_cards, round_name="River")

    print("\n--- Showdown ---")
    hand_values = {
        "Royal Flush": 10, "Straight Flush": 9, "Four of a Kind": 8, "Full House": 7,
        "Flush": 6, "Straight": 5, "Three of a Kind": 4, "Two Pair": 3, "One Pair": 2, "High Card": 1
    }
    print("Community Cards:", community_cards)
    print("\nPlayer Hands and Evaluations:")
    highest_strength = -1
    winner = ""
    best_hand = ""

    for player, hand in hands.items():
        full_hand = hand + community_cards
        hand_strength = evaluate_hand(full_hand)
        strength = hand_values[hand_strength]
        print(f"{player}: {hand} -> {hand_strength}")
        if strength > highest_strength:
            highest_strength = strength
            best_hand = hand_strength
            winner = player

    print(f"\nThe winner is {winner} with a {best_hand}!")
    play_again = input("\nDo you want to play again? (yes/no): ").lower()
    if play_again == "yes":
        play_game()
    else:
        print("Thank you for playing!")
        exit()

# Play the game with 6 players (5 AI players and 1 user)

In [ ]:
play_game()